In [ ]:
def create_random_binary_classifier(category):
    """
    Crea un classificatore binario casuale che predice solo la categoria specificata.

    :param category: La categoria che il classificatore predirà
    :return: Una funzione che genera previsioni casuali della categoria specificata
    """
    def random_binary_classifier(size):
        """
        Genera previsioni casuali della categoria specificata.

        :param size: Numero di previsioni da generare
        :return: Array di previsioni casuali della categoria specificata
        """
        return np.random.choice([0, 1], size=size)
    
    return random_binary_classifier

In [ ]:
for category in categories:
    globals()[f'random_binary_classifier_{category}'] = create_random_binary_classifier(category)

In [ ]:
for category in categories:
    classifier = globals()[f'random_binary_classifier_{category}']
    df_test[f'predicted_{category}'] = classifier(len(df_test))

In [ ]:
print(globals()[f'random_binary_classifier_Openess to change'])

In [ ]:
#creating BERT_w/C classifiers for each category and saving them in the dictionary
for category in categories:
    bert_name = f'BERT_w/C_{category}'
    classifiers[bert_name] = create_bert_classifier(category)

In [ ]:
#apply the BERT w_C classifiers to the test set
for classifier_name, classifier in classifiers.items():
    predictions = classifier(df_test['Conclusion'].tolist())
    df_test[f'predicted_{classifier_name}'] = predictions

print(df_test)

#apply the BERT w_CPS classifiers to the test set WARNING: QUI SERVE DI CONVERTIRE IN LISTA ANCHE STANCE?
for classifier_name, classifier in classifiers.items():
    predictions = classifier(df_test['Conclusion'].tolist(), df_test['Premise'].tolist(), df_test['Stance'].tolist())
    df_test[f'predicted_{classifier_name}'] = predictions

In [ ]:
#creating BERT_w/CP classifiers for each category and saving them in the dictionary
for category in categories:
    bert_name = f'BERT_w/CP_{category}'
    classifiers[bert_name] = create_bert_classifier_with_premise(category)

In [ ]:
#create BERT_w/CPS classifiers for each category and save them in the dictionary
for category in categories:
    bert_name = f'BERT_w/CPS_{category}'
    classifiers[bert_name] = create_bert_classifier_with_premise_and_stance(category)

In [ ]:
# Example usage
categories = ['category1', 'category2']  # Replace with actual categories
y_true = {
    'category': df_test['category'].tolist(),
    'Stance': df_test['Stance_encoded'].tolist()
}
y_pred = [...]  # Replace with predicted labels

# Calculate per-category F1 scores
category_f1_scores = calculate_per_category_f1(y_true, y_pred, categories)

# Calculate average F1 score
average_f1 = calculate_average_f1(category_f1_scores)

print("Per-category F1 scores:", category_f1_scores)
print("Average F1 score:", average_f1)

In [ ]:
    train_dataset = TextDataset(train_texts, train_labels, tokenizer, max_len=128)
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

    # Train the model
    train_bert_classifier(model, train_dataloader)

    # Save the trained model in the dictionary
    bert_name = f'BERT_w/CPS_{category}'
    classifiers[bert_name] = model

In [ ]:
# Training function
def train_bert_classifier(model, dataloader, epochs=3, lr=2e-5):
    optimizer = AdamW(model.parameters(), lr=lr)
    model.train()

    for epoch in range(epochs):
        for batch in dataloader:
            optimizer.zero_grad()
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['label']
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

In [ ]:

    # Create dataset and dataloader for validation
    val_dataset = TextDataset(val_texts, val_labels, tokenizer, max_len=128)
    val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

    # Evaluate the model
    model.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['label']
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    # Calculate per-category F1 scores
    y_true = {
        'category': [category] * len(val_labels),
        'Stance': val_labels
    }
    category_f1_scores = calculate_per_category_f1(y_true, val_preds, [category])

    # Print per-category F1 scores
    print(f"Per-category F1 scores for {category}:", category_f1_scores)

In [ ]:
# Create the data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create the DataLoader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=data_collator)

# Define a simple training loop
def train_model(model, dataloader, epochs=3):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        for batch in dataloader:
            inputs, labels = batch
            optimizer.zero_grad()
            outputs = model(**inputs)
            loss = loss_fn(outputs.logits, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

# Assuming c_model, cp_model, and cps_model are already defined and initialized
train_model(c_model, dataloader)
train_model(cp_model, dataloader)
train_model(cps_model, dataloader)

In [ ]:
for category in categories:

    validation_y_true = df_validation[category]
    test_y_true = df_test[category]
    
    y_pred_BERT_w_c = classifiers[f'BERT_w/C_{category}'](validation_conclusions)
    
    #applying the classifier to the test set
    predictions = classifier(size=len(df_test))
    
    #saving the predictions in the test set
    df_test[f'predicted_{classifier_name}']

In [ ]:
#definition of the model and application to the tokenized output of the Conclusion column

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # Create the DataLoader
    dataloader = DataLoader(df, batch_size=4, shuffle=True, collate_fn=data_collator)

    # Iterate through the DataLoader
    for batch in dataloader:
        inputs, labels = batch
        print(f"Inputs: {inputs}")
        print(f"Labels: {labels}")
        model_c = AutoModelForSequenceClassification.from_pretrained(model_name)
        c_outputs = model_c(**c_inputs)
        c_logits = c_outputs.logits

In [ ]:
#defining per-category F1 score metric
def calculate_per_category_f1(y_true, y_pred, categories):
    category_f1_scores = {}
    for category in categories:
        # Filter true and predicted labels for the current category
        category_indices = [i for i, cat in enumerate(y_true['category']) if cat == category]
        category_y_true = [y_true['Stance'][i] for i in category_indices]
        category_y_pred = [y_pred[i] for i in category_indices]
        
        # Calculate F1 score for the current category
        f1 = f1_score(category_y_true, category_y_pred, average='binary')
        category_f1_scores[category] = f1
    return category_f1_scores

#defining macro F1 score metric
def calculate_macro_f1(category_f1_scores):
    average_f1 = np.mean(list(category_f1_scores.values()))
    return average_f1

In [ ]:
#merge level 2 annotations to level 3 categories

df_test['Openess to change'] = df_test['Self-direction: thought'] | df_test['Self-direction: action'] | df_test['Stimulation'] | df_test['Hedonism'] 
df_test['Self-enhancement'] = df_test['Hedonism'] | df_test['Achievement'] | df_test['Power: dominance'] | df_test['Power: resources'] | df_test['Face']
df_test['Conservation'] = df_test['Face'] | df_test['Security: personal'] | df_test['Security: societal'] | df_test['Tradition'] | df_test['Conformity: rules'] | df_test['Conformity: interpersonal'] | df_test['Humility']
df_test['Self-transcendence'] = df_test['Humility'] | df_test['Benevolence: caring'] | df_test['Benevolence: dependability'] | df_test['Universalism: concern'] | df_test['Universalism: nature']| df_test['Universalism: tolerance'] | df_test['Universalism: objectivity']

df_training['Openess to change'] = df_training['Self-direction: thought'] | df_training['Self-direction: action'] | df_training['Stimulation'] | df_training['Hedonism']
df_training['Self-enhancement'] = df_training['Hedonism'] | df_training['Achievement'] | df_training['Power: dominance'] | df_training['Power: resources'] | df_training['Face']
df_training['Conservation'] = df_training['Face'] | df_training['Security: personal'] | df_training['Security: societal'] | df_training['Tradition'] | df_training['Conformity: rules'] | df_training['Conformity: interpersonal'] | df_training['Humility']
df_training['Self-transcendence'] = df_training['Humility'] | df_training['Benevolence: caring'] | df_training['Benevolence: dependability'] | df_training['Universalism: concern'] | df_training['Universalism: nature']| df_training['Universalism: tolerance'] | df_training['Universalism: objectivity']

df_validation['Openess to change'] = df_validation['Self-direction: thought'] | df_validation['Self-direction: action'] | df_validation['Stimulation'] | df_validation['Hedonism']
df_validation['Self-enhancement'] = df_validation['Hedonism'] | df_validation['Achievement'] | df_validation['Power: dominance'] | df_validation['Power: resources'] | df_validation['Face']
df_validation['Conservation'] = df_validation['Face'] | df_validation['Security: personal'] | df_validation['Security: societal'] | df_validation['Tradition'] | df_validation['Conformity: rules'] | df_validation['Conformity: interpersonal'] | df_validation['Humility']
df_validation['Self-transcendence'] = df_validation['Humility'] | df_validation['Benevolence: caring'] | df_validation['Benevolence: dependability'] | df_validation['Universalism: concern'] | df_validation['Universalism: nature']| df_validation['Universalism: tolerance'] | df_validation['Universalism: objectivity']

df_validation.head()

In [ ]:
model_name = 'bert-base-uncased'
max_length = 100
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenization(ds, stance, model_type):
    # Initialize lists to store the results
    input_ids = []
    attention_masks = []
    token_type_ids = []
 
    if model_type == 'c':
        c_texts = ds['Conclusion']
        
        for text in c_texts:
            encoded_dict = tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=max_len,
                padding='max_length',
                return_token_type_ids=True,
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])
            token_type_ids.append(encoded_dict['token_type_ids'])
            
        # Convert lists to tensors
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)
        token_type_ids = torch.cat(token_type_ids, dim=0)

        # Combine the results into a dictionary
        df_c_inputs = {
            'input_ids': input_ids,
            'attention_mask': attention_masks,
            'token_type_ids': token_type_ids
        }
        return df_c_inputs
    
    elif model_type == 'cp':
        # Extract the list of texts for tokenization of BERT_cp and BERT_cps model inputs
        cp_texts = ds['Conclusion']+[" "]+ds['Premise']

        for text in cp_texts:
            encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens=True,
                        max_length=max_len,
                        padding='max_length',
                        return_token_type_ids=True,
                        truncation=True,
                        return_attention_mask=True,
                        return_tensors='pt'
            )
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])
            token_type_ids.append(encoded_dict['token_type_ids'])

        # Convert lists to tensors
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)
        token_type_ids = torch.cat(token_type_ids, dim=0)

        # Combine the results into a dictionary
        df_cp_inputs = {
            'input_ids': input_ids,
            'attention_mask': attention_masks,
            'token_type_ids': token_type_ids
        }
        return df_cp_inputs   
    
    elif model_type == 'cps': 
        # Extract the list of texts for tokenization of BERT_cp and BERT_cps model inputs
        cps_texts = ds['Conclusion']+[" "]+ds['Premise']
        stance = []

        for text in cps_texts:
            encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens=True,
                        max_length=max_len,
                        padding='max_length',
                        return_token_type_ids=True,
                        truncation=True,
                        return_attention_mask=True,
                        return_tensors='pt'
            )
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])
            token_type_ids.append(encoded_dict['token_type_ids'])

        # Convert lists to tensors
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)
        token_type_ids = torch.cat(token_type_ids, dim=0)
        stance = torch.tensor(stance)

        # Combine the results into a dictionary
        df_cps_inputs = {
            'input_ids': input_ids,
            'attention_mask': attention_masks,
            'token_type_ids': token_type_ids,
            'stance': stance
        }
        return df_cps_inputs   
        

In [ ]:
train_dataset_c = tokenization(ds_training, stance_training, 'c')
train_dataset_cp = tokenization(ds_training, stance_training, 'cp')
train_dataset_cps = tokenization(ds_training, stance_training, 'cps')

val_dataset_c = tokenization(ds_validation, stance_validation, 'c')
val_dataset_cp = tokenization(ds_validation, stance_validation, 'cp')
val_dataset_cps = tokenization(ds_validation, stance_validation, 'cps')

test_dataset_c = tokenization(ds_test, stance_test, 'c')
test_dataset_cp = tokenization(ds_test, stance_test, 'cp')
test_dataset_cps = tokenization(ds_test, stance_test, 'cps')